## Python Modules 

In [2]:
import numpy as np
import matplotlib.pylab as pl

## Modules Created

In [3]:
import parametro
import fortransubroutines as fortran

## Read Bianry File Function

### This function is responsable for reading a binary file 

In [4]:
def readbinaryfile(dim1,dim2,filename):
      """
      readbinaryfile - Functions that read a binary file.
      Usage
      Input:
      dim1     = Number of sample of 1st Dimension
      dim2     = Number of sample of 2nd Dimension
      filename = path of binary file     
      """      
      with open(filename, 'rb') as f:    
            data   = np.fromfile(f, dtype=np.float32, count= dim1*dim2)
            matrix = np.reshape(data, [dim1,dim2], order='F')
      return matrix

## Plot Model

### This function plots the model read in the function above

In [5]:
def plotmodel(matrix,colormap):
      """
      Plot a 2D velocity model read from binary file.

      """      
      pl.figure()
      pl.imshow(matrix,cmap= colormap)

      pl.colorbar()
      pl.draw() # drawing figure to be plotted later

## Plot Graphics

### This function plots 2D graphics: In this project it is used to plot our wavelet and damping function 

In [18]:
def plotgraphics(ID,filename,color):  

      """    
      Plotgraphics -> Function that read a file and plot a graphic.

      REMEMBER: The function 'np.loadtxt' only can be used with files written
            with dot separation

            ID = number of columns of the file    
      """      
      if ID == 2:
            
            X,Y = np.loadtxt(filename, unpack = True)

            pl.figure()
            pl.plot(X,Y, c = color)
            pl.draw()

      if ID == 1:
            
            X = np.loadtxt(filename, unpack = True)

            pl.figure()
            pl.plot(X, c = color)
            pl.draw()
            
      return

## Damping Function

### This python function creates a damping function that will be used in our Cerjan Condition

In [ ]:
def amort(fat_amort,n_grid):
	"""
	amort -> Creates a damping function that will be used in the Cerjan condition.
               The function created is saved in a txt file. 
      
	"""	
	w = np.zeros(n_grid)
  
	for i in np.arange(0,(n_grid)):
		w[i] = np.exp(-(fat_amort * (n_grid-i)) ** 2)  
	  
	np.savetxt('f_amort.dat',w, delimiter='.')

## Source Position 

### This will create our source position in the model we are using

In [ ]:
def posicao_fonte(Nz,Nx,N_shot,Fx0,Fz0,SpaFonte):

      """

      posicao_fonte -> Creates the source positions.
                       The function created is saved in a txt file. 

      """
      
      posicao = np.zeros((N_shot,2))

      Fz = np.arange(0,N_shot)*0 + Fz0
      Fx = np.arange(0,N_shot)*SpaFonte + Fx0


      # if (N_shot*SpaFonte + Fx0) > (Nx -1):
      #    raise ValueError ("Fonte fora do modelo valido (horizontal). Modifique ou o espacamento da fonte ou a posicao inicial Fx0")

      # if (N_shot*SpaFonte + Fz0) > (Nz -1):
      #    raise ValueError ("Fonte fora do modelo valido (vertical). Modifique a posicao inicial Fz0")
     
      posicao[0:N_shot,0] = Fx
      posicao[0:N_shot,1] = Fz

      np.savetxt("posicoes_fonte.dat",posicao,fmt = '%i')

## Modeling Function

### This function is responsable for calling the fortran subroutine where will occur the acoustic modeling. The parameters used are from the 'parametros.py' module. 

In [19]:
def modelagemparalela(shot,\
                      Fx,\
                      Fz,\
                      fonte,\
                      regTTM,\
                      sismograma,\
                      modelo,\
                      nome_prin): 

      """
      This function is responsible for the parallelization of the shots.
      """

      print("Fx =", Fx, "Fz =", Fz, "shot", shot)
      fortran.nucleomodelagem(parametro.Nz,parametro.Nx,parametro.Nt,\
                                    parametro.h,parametro.dt,parametro.nat,\
                                    shot,parametro.shotshow,\
                                    Fx,Fz,fonte,parametro.Nsnap,regTTM,\
                                    modelo,sismograma,\
                                    nome_prin,\
                                    parametro.zr,)
      print(" shot= ",shot," Finalizado.")


## Remove Direct Wave Function

### This calls the fortran function that will remove the direct wave from our seismograms. The parameters used are from the 'parametros.py' module and from our main python file.¶

In [ ]:
def remove_onda_direta(shot,Fx,Fz,nome_prin):

      """
      This function is responsible for the parallelization of the direct wave's removal.
      """
      print("Fx =", Fx, "Fz =", Fz, "shot", shot)
      fortran.removeondadireta(parametro.Nt,parametro.Nx,shot,nome_prin)
      print(" shot= ",shot," Finalizado.")